In [0]:
# import required packages
import numpy as np
import pandas as pd

In [0]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [3]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [0]:
# 1.missing value imputation and change the column(FlightNumber) type from float to int
df.FlightNumber = df.FlightNumber.ffill()+df.FlightNumber.groupby(df.FlightNumber.notnull().cumsum()).cumcount()*10
df.FlightNumber = df.FlightNumber.apply(lambda x: int(x))

In [5]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [6]:
# 2.Splitting "From_To" Column into 2 columns
df_temp = pd.DataFrame(columns = ["From","To"])
df_temp[["From","To"]] = df.From_To.str.split("_",expand=True,)
df_temp

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [7]:
# 3.Standardize Country Names
df_temp = df_temp.apply(lambda x: x.str.capitalize())
df_temp

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


In [8]:
# 4.Delete "From_To" column and merge the df_temp with df
df = df.drop(["From_To"],axis=1)
df = df_temp.merge(df, left_index=True, right_index=True)
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


In [9]:
# 5.To Split Recent Delays values into separate columns
delays = pd.DataFrame(df['RecentDelays'].values.tolist())
delays

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [10]:
# To have the delay column names as delay_1,delay_2 and so on
names = []
new_names =[]
names = list(delays.columns)
for x in names:
  new_names.append("delay_"+str(x+1))
delays.columns = new_names
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [11]:
# Delete "RecentDelays" column and merge the delays with df
df = df.drop(["RecentDelays"],axis=1)
df = df.merge(delays, left_index=True, right_index=True)
df

,From,To,FlightNumber,Airline,delay_1,delay_2,delay_3
0,London,Paris,10045,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055,<Air France> (12),NaN,NaN,NaN
2,London,Stockholm,10065,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075,12. Air France,13.0,NaN,NaN
4,Brussels,London,10085,"""Swiss Air""",67.0,32.0,NaN


In [12]:
# Re-ordering the columns
df = df[['From', 'To', 'FlightNumber', 'delay_1', 'delay_2','delay_3','Airline']]
df


,From,To,FlightNumber,delay_1,delay_2,delay_3,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"""Swiss Air"""
